In [1]:
# ! conda remove torchvision pytorch torchaudio -y

In [2]:
# ! conda install pytorch=2.0.1 torchvision=0.15.2 torchaudio pytorch::torchvision -c pytorch -y

In [3]:
import torch, torchvision
torchvision.disable_beta_transforms_warning()
import sys
import time
import os
import PIL
import pickle
import importlib
import numpy as np

In [4]:
if "__torch_package__" in dir():
    #inside a torch package
    import torch_package_importer
    import_func = torch_package_importer.import_module 
else:
    #normal
    import importlib
    import_func = lambda m: importlib.reload(importlib.import_module(m))

#internal modules
MODULES = ['datasets', 'traininglib']
[datasets, traininglib] = [import_func(m) for m in MODULES]

from modellib import DuckDetector

def save(self, destination):
    if isinstance(destination, str):
        destination = time.strftime(destination)
        if not destination.endswith('.pt.zip'):
            destination += '.pt.zip'
    try:
        import torch.package.package_importer as imp
        #re-export
        importer = (imp, torch.package.sys_importer)
    except ImportError as e:
        #first export
        importer = (torch.package.sys_importer,)

    with torch.package.PackageExporter(destination, debug = True) as exp:
        # interns = [__name__.split('.')[-1]]+MODULES
        interns = [__name__.split('.')[-1]]+MODULES
        exp.intern(interns)
        exp.extern('**', exclude=['torchvision.**'])
        externs = ['torchvision.ops.**', 'torchvision.datasets.**', 'torchvision.io.**']
        exp.intern('torchvision.**', exclude=externs)
        exp.extern(externs)
        exp.intern('torchvision.models.detection.**')
        # force inclusion of internal modules + re-save if importlib.reload'ed
        for m in MODULES:
            exp.save_module(m, import_func(m))
        exp.save_module('modellib')
        exp.save_pickle('model', 'model.pkl', self)
        exp.save_text('model', 'class_list.txt', '\n'.join(self.class_list))
    return destination

In [5]:
basemodel_pt_zip = save(self=DuckDetector(classes_of_interest=['',
                                    'Fulica americana',
                                    'Mareca strepera',
                                    'Anas carolinensis',
                                    'Anas acuta',
                                    'Spatula clypeata',
                                    'Anas platyrhynchos',
                                    'Aythya americana',
                                    'Aythya collaris']), 
                                    destination="basemodel.pt.zip")

In [6]:
imp = torch.package.PackageImporter(basemodel_pt_zip)
print(imp.file_structure())

─── basemodel.pt.zip
    ├── .data
    │   ├── 0.storage
    │   ├── 1.storage
    │   ├── 10.storage
    │   ├── 11.storage
    │   ├── 12.storage
    │   ├── 13.storage
    │   ├── 14.storage
    │   ├── 15.storage
    │   ├── 16.storage
    │   ├── 17.storage
    │   ├── 18.storage
    │   ├── 19.storage
    │   ├── 2.storage
    │   ├── 20.storage
    │   ├── 21.storage
    │   ├── 22.storage
    │   ├── 23.storage
    │   ├── 24.storage
    │   ├── 25.storage
    │   ├── 26.storage
    │   ├── 27.storage
    │   ├── 28.storage
    │   ├── 29.storage
    │   ├── 3.storage
    │   ├── 30.storage
    │   ├── 31.storage
    │   ├── 32.storage
    │   ├── 33.storage
    │   ├── 34.storage
    │   ├── 35.storage
    │   ├── 36.storage
    │   ├── 37.storage
    │   ├── 38.storage
    │   ├── 39.storage
    │   ├── 4.storage
    │   ├── 40.storage
    │   ├── 41.storage
    │   ├── 42.storage
    │   ├── 43.storage
    │   ├── 44.storage
    │   ├── 45.storage
    │   ├── 46.storage
    

### <center>Test that packaged model opens</center>

In [7]:
def load_modelfile(file_path:str) -> "torch.nn.Module":
        if file_path.endswith('.pt.zip'):
            return torch.package.PackageImporter('basemodel.pt.zip').load_pickle('model', 'model.pkl', map_location='cpu')
        elif file_path.endswith('.pkl'):
            import pickle
            return pickle.load(open('basemodel.pt.zip', 'rb'))

In [8]:
file_path = 'basemodel.pt.zip'
model = load_modelfile(file_path=file_path)


In [9]:
print(model)

DuckDetector(
  (detector): Detector(
    (basemodel): SSD(
      (backbone): SSDFeatureExtractorVGG(
        (features): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU(inplace=True)
          (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (3): ReLU(inplace=True)
          (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (6): ReLU(inplace=True)
          (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (8): ReLU(inplace=True)
          (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (11): ReLU(inplace=True)
          (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (13): ReLU(

### <center> Run model on sample images </center>

In [10]:
#import datasets module from datasets.py
datasets = import_func('datasets')
modellib = import_func('modellib')


In [11]:
# create dataset
sample_dataset = datasets.DetectionDataset(jpgfiles='C:Users/zack/Desktop/DuckNet_Data_Test/Images/', jsonfiles='C:Users/zack/Desktop/DuckNet_Data_Test/Annotations/', augment=False, negative_classes=None, image_size=300)

In [12]:
# create dataloader
sample_dataloader = datasets.create_dataloader(sample_dataset, batch_size=1, shuffle=False)

In [13]:
# how many images in sample dataloader
print(len(sample_dataloader))

46


In [14]:
print(model.class_list)

['', 'Fulica americana', 'Mareca strepera', 'Anas carolinensis', 'Anas acuta', 'Spatula clypeata', 'Anas platyrhynchos', 'Aythya americana', 'Aythya collaris']


In [15]:
print(model.load_image('C:/Users/zack/Desktop/DuckNet_Data_Test/Images/DJI_20211215103949_0003_Z.JPG'))

<PIL.Image.Image image mode=RGB size=5184x2916 at 0x2213A002820>


In [16]:
prediction = model.process_image(model.load_image('C:/Users/zack/Desktop/DuckNet_Data_Test/Images/DJI_20211215103949_0003_Z.JPG'))

In [17]:
# print prediction
print(prediction)


{'boxes': array([[2076.3042   , 1862.7476   , 2739.5593   , 2340.5957   ],
       [ 627.97546  , 1543.8413   , 1306.377    , 1952.0632   ],
       [   0.       , 1326.8381   ,  228.88193  , 1689.2069   ],
       [ 115.8404   , 1397.0332   ,  633.07764  , 1901.1962   ],
       [2737.5579   , 1457.5819   , 2887.8318   , 1757.4143   ],
       [1006.38214  , 1254.3647   , 2874.5383   , 2157.5      ],
       [2679.705    , 2333.2283   , 3104.4763   , 2881.396    ],
       [ 450.7682   , 1452.4226   , 1061.0837   , 1926.9851   ],
       [3402.3733   , 1891.5674   , 4674.0107   , 2427.7102   ],
       [2028.096    , 1638.1885   , 2695.8147   , 2039.7543   ],
       [ 898.84314  , 1308.8619   , 2085.082    , 1940.3439   ],
       [  47.534107 , 1321.4209   ,  957.33295  , 1959.8663   ],
       [2690.9238   , 1509.4287   , 3170.4858   , 2111.0554   ],
       [3008.5354   , 1651.2396   , 3137.1235   , 2038.7169   ],
       [1857.7765   , 1182.8284   , 3056.9368   , 2328.5144   ],
       [ 496.25